In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Path to Dataframe
path = r'C:\\Users\\maad0\\OneDrive\\CAREERFOUNDRY NOTES\\Python Fundamentals for Data Analyst\\08-2023 Instacart Basket Analysis'

In [3]:
# Import orders_products_merged_v4
df_ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_v4.pkl'))

### Question 2.  In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [4]:
# Aggregating mean of entire dataframe
df_ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.213779
6,16.439806
7,17.225802
8,15.340650


### Question 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

#### The subset of the merged dataframe provides a subset of the rows in the dataframe, whereas the results from the entire dataframe show the ‘mean’ of all departments.\n",
    "#### Additionally, the average of the orders per department is higher when searching a subset of the dataframe as opposed to searching the mean of the entire dataframe, i.e., the subset mean of Dept 4 is ‘19’, the average goes down to ‘18’ when searching the entire dataframe.

### Question 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [5]:
# Aggregating Data with transform() function
df_ords_prods_merged['max_order'] = df_ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
# Checking results
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,orders_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average Orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average Orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most Orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average Orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most Orders,10


### Deriving Columns with loc()

In [7]:
# Creating Flag that assigns a loyalty flag orders > 40
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
# Creating Flag that assigns loyalty flag orders <= 40 and >10
df_ords_prods_merged.loc[(df_ords_prods_merged['max_order'] <= 40) & (df_ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [9]:
# Creating Flag that assigns loyalty flag orders <= 10
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
# Print the frequency of new “loyalty_flag” columns using the value_counts() function:
df_ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15877395
Loyal customer      10284396
New customer         6244250
Name: loyalty_flag, dtype: int64

### Question 5.  The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [11]:
# Performing Multiple Aggregations to check the basic statistics of the product prices for each loyalty category
df_ords_prods_merged.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386229  1.0  99999.0
New customer      13.294406  1.0  99999.0
Regular customer  12.495501  1.0  99999.0

####  Observation of the basic statistics:\n",
#### The first thing that catches my eye is the maximum order cap of 99,999.  This number appears too high and may have been a data-entry error.  In reviewing the mean for each loyalty category, the average prices of products purchased by Loyal customers are less than those of the other two categories, with the New customers having the higher prices in the loyalty category. 

### Question 6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
## If the mean of the prices of products purchased by a user is lower than 10,  then flag them as a “Low spender.
## If the mean of the prices of products purchased by a user is higher than or equal to 10,   then flag them as a “High spender.

In [12]:
# Checking Column names
df_ords_prods_merged.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'orders_day_of_week',
       'orders_time_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', '_merge', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag'],
      dtype='object')

In [13]:
# Applying transform function on \"user_id\" column
df_ords_prods_merged['avg_price'] = df_ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [14]:
# Checking Results
df_ords_prods_merged.head(10)

,order_id,user_id,eval_set,order_number,orders_day_of_week,orders_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average Orders,10,New customer,6.367797
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average Orders,10,New customer,6.367797
2,473747,1,prior,3,3,12,21.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most Orders,10,New customer,6.367797
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Slowest days,Average Orders,10,New customer,6.367797
4,431534,1,prior,5,4,15,28.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Slowest days,Most Orders,10,New customer,6.367797
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average Orders,10,New customer,6.367797
6,550135,1,prior,7,1,9,20.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most Orders,10,New customer,6.367797
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most Orders,10,New customer,6.367797
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most Orders,10,New customer,6.367797
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Slowest days,Average Orders,10,New customer,6.367797


In [15]:
# Creating Spending Flag for Users
df_ords_prods_merged.loc[df_ords_prods_merged['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [16]:
df_ords_prods_merged.loc[df_ords_prods_merged['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [17]:
df_ords_prods_merged['spending_flag'].value_counts(dropna = False)

Low spender     31771789
High spender      634252
Name: spending_flag, dtype: int64

In [18]:
#Checking Results
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,orders_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average Orders,10,New customer,6.367797,Low spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Average Orders,10,New customer,6.367797,Low spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Most Orders,10,New customer,6.367797,Low spender
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Slowest days,Average Orders,10,New customer,6.367797,Low spender
4,431534,1,prior,5,4,15,28.0,196,1,1,...,9.0,both,Mid-range product,Least busy,Slowest days,Most Orders,10,New customer,6.367797,Low spender


### Question 7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
#### If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.
#### If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
#### If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [19]:
# Applying transform function on \"user_id\" column
df_ords_prods_merged['median_prior_orders'] = df_ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
# Creating Order Frequency Flag for Users
df_ords_prods_merged.loc[df_ords_prods_merged['median_prior_orders'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [21]:
df_ords_prods_merged.loc[df_ords_prods_merged['median_prior_orders'] > 10 & (df_ords_prods_merged['median_prior_orders'] <=20), 'frequency_flag'] = 'Regular customer'

In [22]:
df_ords_prods_merged.loc[df_ords_prods_merged['median_prior_orders'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [23]:
# Checking results
df_ords_prods_merged['frequency_flag'].value_counts(dropna = False)

Frequent customer    21560324
Regular customer     10845712
NaN                         5
Name: frequency_flag, dtype: int64

In [24]:
# Dataframe check
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,orders_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_prior_orders,frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Regular customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Average Orders,10,New customer,6.367797,Low spender,20.5,Regular customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Most Orders,10,New customer,6.367797,Low spender,20.5,Regular customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy,Slowest days,Average Orders,10,New customer,6.367797,Low spender,20.5,Regular customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy,Slowest days,Most Orders,10,New customer,6.367797,Low spender,20.5,Regular customer


In [25]:
# Exporting updated dataframe
df_ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_v5.pkl'))